# Recategorize score
 Rescore model probabilities using Buckets from train. Saves results to specified database table

## Load libraries

In [61]:
pip install tabgan==1.3.3

Note: you may need to restart the kernel to use updated packages.


In [62]:
!pip install lightgbm

In [63]:
pip install scikit-plot

Note: you may need to restart the kernel to use updated packages.


In [64]:
%pip install scipy==1.11.4

Note: you may need to restart the kernel to use updated packages.


In [65]:
pip install --upgrade pandas "dask[complete]"

Note: you may need to restart the kernel to use updated packages.


In [66]:
%pip install scikit-learn==1.1.3

Note: you may need to restart the kernel to use updated packages.


In [67]:
## librerias requeridas
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import re
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_lift_curve
from scikitplot.helpers import binary_ks_curve 
from scikitplot.metrics import plot_ks_statistic
from scikitplot.helpers import cumulative_gain_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from datetime import datetime, timedelta

import pyarrow as pa
import pyarrow.parquet as pq

In [68]:
import sklearn
print(sklearn.__version__)

1.1.3


In [69]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.memory_usage', True)

## Load model and data to evaluate

Load variables names from file

In [70]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/viamericas-dev/Analytics/5.Fraud'

In [71]:
import os
import joblib
# load model
modelo = joblib.load("ModeloAWSRetrain.pkl") 
variables = open('./ModeloBiteRetrain.txt','r')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [72]:
modelo

LGBMClassifier(colsample_bytree=0.84, importance_type='gini',
               learning_rate=0.07, max_depth=20, min_child_weight=0.0026,
               n_estimators=520, num_leaves=30, objective='binary',
               scale_pos_weight=0.003, subsample=1)

## Veo si puedo sacar los nombres directo del modelo q cargue
## Veo como cargar el punto de corte

In [73]:
variables = open('./ModeloBiteRetrain.txt','r')
var_input = variables.read()

#1er prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','01_isMexico', 
    '01_sender_branch_state', '01_var_range_hist', 
    '01_branch_fraud', '01_location_fraud', '01_sender_fraud', 
    '01_isCashPick', '01_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    '01_receiver_transaction_count','01_sender_sending_days',
    '01_branch_working_days', '01_net_amount_receiver', 
    '01_sender_minutes_since_last_transaction_2days', 
    '01_sender_days_to_last_transaction_365', 
    '01_sender_days_to_last_transaction_7m', '01_hour_receiver']

In [74]:
#2da prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','01_isMexico', 
    '01_sender_branch_state', '01_var_range_hist', 
    '01_branch_fraud', '01_location_fraud',
    '01_isCashPick', '01_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    'receiver_transaction_count','01_sender_sending_days',
    'branch_working_days', '01_net_amount_receiver', 
    'sender_minutes_since_last_transaction_2days', 
    'sender_days_to_last_transaction_365', 
    'sender_days_to_last_transaction_7m', '01_hour_receiver']

In [76]:
s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodosRetrain"
#tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Faltantes.parquet")

## Variable list for input
X = tes[var_input]
Y = tes[['target']]

dias_target = ['day_name_receiver_0', 'day_name_receiver_1',
       #'day_name_receiver_2', 'day_name_receiver_3',
       #'day_name_receiver_4', 'day_name_receiver_5',
       'day_name_receiver_6','target']

testeo =  tes.copy()
Y_target = pd.DataFrame(tes[dias_target])



In [ ]:
#s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos"
#tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

#testeo = pd.concat([testeo, tes], axis = 0, ignore_index=True)
#Y_target = pd.DataFrame(pd.concat([Y_target, tes[dias_target]], ignore_index=True)) 

## Apply model

In [77]:
# Prediciendo la variable objetivo para el conjunto de datos de prueba 'X_TESTEO' utilizando el clasificador LightGBM
testeo_data_pred_rf = modelo.predict(testeo[var_input])

# Prediciendo las probabilidades de clase para cada observación en 'X_TESTEO' utilizando el clasificador LightGBM
probab_rf = modelo.predict_proba(testeo[var_input])

# Eliminando la primera columna (se asume que son las probabilidades de la clase negativa) de las probabilidades predichas
# Se asume que la segunda columna contiene las probabilidades para la clase positiva
score_rf = np.delete(probab_rf, np.s_[0], axis=1)


In [78]:
# Creando una copia del DataFrame 'Y_TESTEO' para preservar los datos originales
Y_c = Y_target.copy()

# Agregando una nueva columna 'preds_rf' al DataFrame copiado con las predicciones del clasificador LightGBM
Y_c['preds_rf'] = testeo_data_pred_rf

# Agregando una nueva columna 'score_rf' al DataFrame copiado con las probabilidades de la clase positiva
Y_c['score_rf'] = score_rf

In [79]:
pd.DataFrame(confusion_matrix(Y_c['target'], Y_c['preds_rf']))

,0,1
0,489192,0
1,5,0


In [80]:
giro = testeo[var_input]
giro = np.array(giro.iloc[1])
giro
giro_score = modelo.predict_proba(giro.reshape(1, -1))

## Transformo los datos para subirlos a Athena

In [81]:
Buckets = pd.read_csv('./BucketsModeloAWSRetrain.csv')

In [82]:
# Create bins from reference dataframe
pd.options.display.float_format = "{:,.15f}".format
# Convert data types of 'Min_Value' and 'Max_Value' to numeric
Buckets['min'] = pd.to_numeric(Buckets['min'])
Buckets['max'] = pd.to_numeric(Buckets['max'])
Y_c['score_rf'] = pd.to_numeric(Y_c['score_rf'])

#bins = np.sort(np.append(Buckets['min'], Buckets['max'].iloc[-1]))
bins = np.sort(np.append(Buckets['min'], np.inf))  

labels = np.sort(Buckets['final_score'])
#labels = np.sort(1000-Buckets['FRAUD_DECILE'])

# Recode scores using pd.cut()
Y_c['score_AWS'] = pd.cut(Y_c['score_rf'], bins=bins, labels=labels) ##, include_lowest=True, right=False)
#Y_c['Recode'] = Y_c['Recode'].astype('Int32')

Y_c.head()

,day_name_receiver_0,day_name_receiver_1,day_name_receiver_6,target,preds_rf,score_rf,score_AWS
0,True,False,False,False,False,0.000000000024926,0.513000000000000
1,False,False,False,False,False,0.000000000014730,0.427000000000000
2,False,False,False,False,False,0.000000000090944,0.703000000000000
3,True,False,False,False,False,0.000000000068265,0.665000000000000
4,False,False,False,False,False,0.000000000003583,0.212000000000000


In [83]:
Y_c['pred_corte_01'] = Y_c['score_AWS']>0.998 

In [84]:
Y_c['pred_corte_01'].value_counts()

pred_corte_01
False    488726
True        471
Name: count, dtype: int64

In [85]:
#See false positives by month
tes_corte = testeo[Y_c['pred_corte_01']==True]
Y_c_corte = Y_c[Y_c['pred_corte_01']==True]

pd.crosstab(tes_corte['date_receiver'].dt.day, Y_c_corte['target'])

target,False,True
date_receiver,,
25,76,0
26,94,2
27,76,0
28,95,0
29,66,1
30,61,0


In [86]:
tes_corte[tes_corte['date_receiver'].dt.day==30]

,id_branch,id_receiver,id_flag_receiver,id_main_branch_sent,source,transaction_unique,date_receiver,hour_receiver,minute_to_payment,status,was_fraud,fraud_classification,fraud_incident_code,address_receiver_claim,id_country_receiver_claim,id_state_receiver_claim,id_city_receiver_claim,name_receiver,phone1_receiver,phone2_receiver,id_payer,receiver_unique,id_sender_global,id_country_issuer_id,id_country_issuer_passport,occupation_sender,inserted_date,installation_type,name_branch,id_city,id_state,zip_branch,id_chain,id_location,phone1_branch,phone2_branch,tipo_transmision,branch_working_days,sender_sending_days,sender_days_to_last_transaction,was_fraud_pattern,name_sender,addres_sender,city_sender,id_country,phone1_sender,phone2_sender,sender_days_since_last_transaction,sender_minutes_since_last_transaction,receiver_transaction_count,receiver_date_first_transaction,receiver_date_last_transaction,branch_minutes_since_last_transaction,was_ofac_sender_hit,was_ofac_receiver_hit,was_false_positive,net_amount_receiver,fraud_classification_abt,bank_receiver_abt,branch_trans_3m,sender_state,day,branch_has_fraud,receiver_has_fraud,branch_trans_40min,branch_trans_10min,cash_pick_up_40min,location_nro_fraud,sender_trans_3m,range_hist,sender_nro_fraud,row,branch_trans_3m_correct,mes,01_isMexico,01_sender_branch_state,01_var_range_hist,01_receiver_fraud,01_branch_fraud,01_location_fraud,01_sender_fraud,01_isCashPick,01_isBankDep,sender_days_to_last_transaction_365,sender_days_to_last_transaction_more365,sender_days_to_last_transaction_7m,sender_days_to_last_transaction_more7m,sender_minutes_since_last_transaction_2days,sender_minutes_since_last_transaction_1day,01_net_amount_receiver,01_receiver_transaction_count,01_sender_sending_days,01_sender_minutes_since_last_transaction_2days,01_sender_minutes_since_last_transaction_1day,01_sender_days_to_last_transaction_365,01_sender_days_to_last_transaction_7m,01_hour_receiver,day_name_receiver_0,day_name_receiver_1,day_name_receiver_2,day_name_receiver_3,day_name_receiver_5,day_name_receiver_6,id_payout_A,id_payout_C,id_payout_D,id_payout_G,id_payout_M,id_payout_N,id_payout_O,id_payout_P,id_payout_S,id_payout_T,id_payout_X,targetR,targetP,target
3970,A26401,1481,P,T283,O,A26401-1481,2024-05-30 19:39:55.007,19,30,Paid,0,0,0,0,MEX,MEX12,ME580,b'\xd4\x8bW\xc0BD)\x99\xa4\xc0\xdd\x14\x0e\xf8...,b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x...,b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x...,T283,b'XJ\x11\x8f\x9a\x16 t\x069e\x9b\xaa\x07\xf8+',11946207,USA,,,2024-05-30 19:36:52.643,None,INTERNATIONAL MULTISERVICES (TUCKER),02014,GA,30084,USA021454,GA1301,6786941575,None,VO,83,0,0,0,b'@zl\x18\xbc\x9c$\x917\x1f\xeb\x85d<d\x18',b'o\x83\xd1\xc0\xd2\x0e\xd8$\xa0\x96BO<\xb0\xf...,01472,USA,b'5Xj\x05@#\xf8\xc7\xf4\x00\x1c\xfeC\x81YK',None,0,0,1,2024-05-30,2024-05-30,54,0,0,None,990,None,None,"1,416.000000000000000",GA,2024-05-30,0,0,0,0,NaN,0,0,NaN,0,1,"1,416.000000000000000",5,True,True,NaN,False,False,False,False,False,True,0,0,0,0,0,0,0.357400722021661,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.791666666666667,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
15537,A26233,94,P,T239,O,A26233-94,2024-05-30 14:31:10.437,14,13,Paid,0,0,0,0,MEX,MEX27,MT127,b'\xb5\x12\xf0\x00\x1b\xb4t\x7f\x88\\\xe1\xbed...,b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x...,b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x...,T239,b'?_Y\xa7B\x03\xe7\x14\xdfY\x90\xbdJ3\xd7\xb8',8074792,USA,net.razorvine.pickle.objects.ClassDictConstruc...,,2024-05-30 14:31:10.187,None,EL SUPER (MONTROSE),81401,CO,81401,USA015532,CO1380,9704175331,None,VO,104,0,0,0,b'tp\x10\xa3\x86Yi:i\xd5\x13\x8e\xb3\xa9\xf0\xda',b'\x14\xdfXEW\xb1\x08\x92zh\x17\xfa\x1e%V\xd1',81401,USA,"b""\x8aE\x1df\x0c\x1f\x00\x10\xd6\x8f\x19't\xc5...",None,0,0,1,2024-05-30,2024-05-30,2565,0,0,None,970,None,None,84.000000000000000,CO,2024-05-30,0,0,0

In [87]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
df = pd.concat([testeo[['id_branch','id_receiver']], Y_c[['score_AWS','pred_corte_01']]], axis=1) 
df = df.rename(columns={"pred_corte_01": "was_fraud_AWS"})

## Guardo en Athena

In [ ]:
pip install awswrangler

In [ ]:
df

In [ ]:
import awswrangler as wr
import pandas as pd
import boto3
import time
from time import sleep

# Reemplazar aca por el dataframe a guardar
df = df

wr.s3.to_parquet(
    df=df,
    path='s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws/',
    dataset=True,
    mode="overwrite"
)

#lambda_crawler
def wait_for_crawler_completion(crawler_name: str, poll_interval: int = 15) -> str:
	"""
	Waits until Glue crawler completes and
	returns the status of the latest crawl run.
	Raises AirflowException if the crawler fails or is cancelled.
	:param crawler_name: unique crawler name per AWS account
	:param poll_interval: Time (in seconds) to wait between two consecutive calls to check crawler status
	:return: Crawler's status
	"""
	client = boto3.client('glue')
	response = client.start_crawler(Name=crawler_name)
	failed_status = ['FAILED', 'CANCELLED']
	sleep(5)

	while True:
		crawler = client.get_crawler(Name=crawler_name)
		crawler_state = crawler['Crawler']['State']
		if 'LastCrawl' in crawler['Crawler']:
			if crawler_state == 'STOPPING' or crawler_state == 'READY':
				print("State: %s", crawler_state)
				print("crawler_config: %s", crawler)
				crawler_status = crawler['Crawler']['LastCrawl']['Status']
				if crawler_status in failed_status:
						raise Exception(f"Status: {crawler_status}")
				metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
				print("Status: %s", crawler_status)
				print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
				print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
				print("Tables Created: %s", metrics['TablesCreated'])
				print("Tables Updated: %s", metrics['TablesUpdated'])
				print("Tables Deleted: %s", metrics['TablesDeleted'])

				return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)
		else:
			if crawler_state == 'READY':
					print("State: %s", crawler_state)
					print("crawler_config: %s", crawler)
					crawler_status = crawler['Crawler']['LastCrawl']['Status']
					if crawler_status in failed_status:
							raise Exception(f"Status: {crawler_status}")
					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
					print("Status: %s", crawler_status)
					print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
					print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
					print("Tables Created: %s", metrics['TablesCreated'])
					print("Tables Updated: %s", metrics['TablesUpdated'])
					print("Tables Deleted: %s", metrics['TablesDeleted'])

					return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)


CrawlerName = 'crw_fraud_model_input'
crawler_to_run = f'{CrawlerName}'
print('Running Crawler: ', crawler_to_run)
response = wait_for_crawler_completion(crawler_name=crawler_to_run)
print('Finish with status: ', response)